In [11]:
import pandas as pd
import numpy as np

In [12]:
employers = pd.read_csv('Attrition_dataset.csv', sep=';')
cities = pd.read_csv('Cities_map.csv', sep=';')

In [13]:
print(employers.head())

   Emp_ID      MMM.YY  Age        Salary LastWorkingDate  Designation  \
0       1  01.01.2016   28  57387,00 USD             NaN            1   
1       1  01.02.2016   28  57387,00 USD             NaN            1   
2       1  01.03.2016   28  57387,00 USD      11.03.2016            1   
3       2  01.11.2017   31  67016,00 USD             NaN            2   
4       2  01.12.2017   31  67016,00 USD             NaN            2   

   Total.Business.Value  Quarterly.Rating Gender City Education_Level  \
0               2381060                 2   Male  C23          MASTER   
1               -665480                 2   Male  C23          MASTER   
2                     0                 2   Male  C23          MASTER   
3                     0                 1   Male   C7          Master   
4                     0                 1   Male   C7          Master   

  Dateofjoining  Joining.Designation  
0    24.12.2015                    1  
1    24.12.2015                    1  
2    

In [14]:
print(cities.head())

  City_ID  City_Name
0     C01     Austin
1     C02     Boston
2     C03  Charlotte
3     C04    Chicago
4     C05   Columbus


In [15]:
employers.rename(columns={
    'Emp_ID': 'ID',
    'MMM.YY': 'Day of Month Rate',
    'LastWorkingDate': 'Last Working Date',
    'Total.Business.Value': 'Total Business Value',
    'Quarterly.Rating': 'Quarterly Rating',
    'Education_Level': 'Education Level',
    'Dateofjoining': 'Date of Joining',
    'Joining.Designation':'Joining Designation'
}, inplace=True)

In [16]:
print(employers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19190 entries, 0 to 19189
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    19190 non-null  int64 
 1   Day of Month Rate     19190 non-null  object
 2   Age                   19190 non-null  int64 
 3   Salary                19190 non-null  object
 4   Last Working Date     1628 non-null   object
 5   Designation           19190 non-null  int64 
 6   Total Business Value  19190 non-null  int64 
 7   Quarterly Rating      19190 non-null  int64 
 8   Gender                19190 non-null  object
 9   City                  19190 non-null  object
 10  Education Level       19190 non-null  object
 11  Date of Joining       19190 non-null  object
 12  Joining Designation   19190 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 1.9+ MB
None


In [17]:
employers.describe(include='all')

,ID,Day of Month Rate,Age,Salary,Last Working Date,Designation,Total Business Value,Quarterly Rating,Gender,City,Education Level,Date of Joining,Joining Designation
count,19190.000000,19190,19190.000000,19190,1628,19190.000000,1.919000e+04,19190.000000,19190,19190,19190,19190,19190.000000
unique,NaN,24,NaN,2383,493,NaN,NaN,NaN,2,29,6,869,NaN
top,NaN,01.01.2016,NaN,"48747,00 USD",29.07.2017,NaN,NaN,NaN,Male,C20,Bachelor,23.07.2012,NaN
freq,NaN,1025,NaN,57,71,NaN,NaN,NaN,11151,1014,5551,194,NaN
mean,1416.239083,NaN,34.649192,NaN,NaN,2.252579,5.710257e+05,2.008963,NaN,NaN,NaN,NaN,1.690412
std,810.866259,NaN,6.264600,NaN,NaN,1.026329,1.126591e+06,1.009839,NaN,NaN,NaN,NaN,0.836580
min,1.000000,NaN,21.000000,NaN,NaN,1.000000,-6.000000e+06,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,712.000000,NaN,30.000000,NaN,NaN,1.000000,0.000000e+00,1.000000,NaN,NaN,NaN,NaN,1.000000
50%,1417.500000,NaN,34.000000,NaN,NaN,2.000000,2.500000e+05,2.000000,NaN,NaN,NaN,NaN,1.000000
75%,2139.000000,NaN,39.000000,NaN,NaN,3.000000,6.995850e+05,3.000000,NaN,NaN,NaN,NaN,2.000000


In [18]:
print(employers['Education Level'])

0            MASTER
1            MASTER
2            MASTER
3            Master
4            Master
            ...    
19185        Colege
19186        Master
19187      Bachelor
19188    Bachelor's
19189        Master
Name: Education Level, Length: 19190, dtype: object


In [19]:
employers['Education Level'] = employers['Education Level'].apply(lambda x: 'Master' if x[0]== 'M' else ('College' if x[0]=='C' else 'Bachelor'))

In [20]:
print(employers['Education Level'])

0          Master
1          Master
2          Master
3          Master
4          Master
           ...   
19185     College
19186      Master
19187    Bachelor
19188    Bachelor
19189      Master
Name: Education Level, Length: 19190, dtype: object


In [21]:
employers['Education Level'].nunique()

3

In [27]:
df = pd.merge(employers, cities.rename(
columns={
    'City_ID': 'City'
}))

In [28]:
print(df)

         ID Day of Month Rate  Age        Salary Last Working Date  \
0         1        01.01.2016   28  57387,00 USD               NaN   
1         1        01.02.2016   28  57387,00 USD               NaN   
2         1        01.03.2016   28  57387,00 USD        11.03.2016   
3        12        01.07.2016   35  28116,00 USD               NaN   
4        12        01.08.2016   35  28116,00 USD               NaN   
...     ...               ...  ...           ...               ...   
13639  1763        01.01.2017   22  33042,00 USD        21.01.2017   
13640  1161        01.10.2017   30  67371,00 USD               NaN   
13641  1746        01.01.2016   49  38478,00 USD               NaN   
13642  1043        01.05.2017   33  93121,00 USD               NaN   
13643  1606        01.09.2016   43  99127,00 USD               NaN   

       Designation  Total Business Value  Quarterly Rating  Gender City  \
0                1               2381060                 2    Male  C23   
1        

In [30]:
df.drop('City', inplace=True, axis=1)

In [32]:
df.rename(columns={
    'City_Name': 'City'
}, inplace=True)

In [33]:
print(df.head())

   ID Day of Month Rate  Age        Salary Last Working Date  Designation  \
0   1        01.01.2016   28  57387,00 USD               NaN            1   
1   1        01.02.2016   28  57387,00 USD               NaN            1   
2   1        01.03.2016   28  57387,00 USD        11.03.2016            1   
3  12        01.07.2016   35  28116,00 USD               NaN            1   
4  12        01.08.2016   35  28116,00 USD               NaN            1   

   Total Business Value  Quarterly Rating Gender Education Level  \
0               2381060                 2   Male          Master   
1               -665480                 2   Male          Master   
2                     0                 2   Male          Master   
3                500000                 4   Male          Master   
4               1707180                 4   Male          Master   

  Date of Joining  Joining Designation      City  
0      24.12.2015                    1  Portland  
1      24.12.2015         